### What's in our backpack

Here's a list of functions packing up what we've learnt in the previous lesson.

In [ ]:
def argmax(arr):
    """Find the maximum point of a 1-d or 2-d array"""
    if len(arr.shape) == 1:
        return np.argmax(arr)
    else:
        return np.unravel_index(np.argmax(arr), arr.shape)
    
def get_data(name):
    """Get data from a FITS frame"""
    hdul = fits.open(name)
    return hdul[0].data

def get_header(name):
    """Get header from a FITS frame"""
    hdul = fits.open(name)
    return hdul[0].header

def plot(arr, vmin=None, vmax=None, argmax=None, new=True, **kwargs):
    """Plot a 1-d or 2-d array"""
    if len(arr.shape) == 1: 
        plot_1d(arr, argmax, new, **kwargs)
    elif len(arr.shape) == 2:
        plot_2d(arr, vmin, vmax, argmax, new, **kwargs)
    else:
        pass
    
def plot_1d(arr, argmax=None, new=True, xlabel='pixel', ylabel='ADU', **kwargs):
    """Plot a 1-d array, possibly with its maximum highlighted"""
    if new: plt.figure(figsize=(16,8))
    plt.plot(range(len(arr)), arr, **kwargs)
    if argmax is not None: plt.scatter(argmax, np.max(arr), color='r')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

def plot_2d(arr, vmin=None, vmax=None, argmax=None, new=True, xlabel='x pixel', ylabel='y pixel', **kwargs):
    """Plot a 2-d array, possibly with its maximum highlighted"""
    if new: plt.figure(figsize=(16,8))
    plt.imshow(arr, vmin=vmin, vmax=vmax, **kwargs)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.colorbar()
    if argmax is not None: plt.scatter(argmax[1], argmax[0], color='r')
    
def plot_medrange(arr, hwid=200, argmax=None, new=True, **kwargs):
    """Same as `plot`, but with a colorbar centered around the median value"""
    plot(arr, np.nanmedian(arr)-hwid, np.nanmedian(arr)+hwid, argmax, new, **kwargs)
    
def plot_series(t, f, new=True, xlabel='time', ylabel='Flux (ph)', **kwargs):
    if new: plt.figure(figsize=(16,8))
    plt.plot(t, f, **kwargs)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

### Get rid of bias

We can easily get an idea of the bias by setting a proper range for the colorbar when plotting our science frames:

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
    
names = np.array(['/Users/guido/Dropbox/courses/2019/SZ_Lyn/Autosave Image -%03i.fit' % i for i in range(1,211)])
sci = np.array([get_data(n) for n in names])
sci_hdr = np.array([get_header(n) for n in names])
plot_medrange(sci[0])  # Only 1st frame is plotted
plt.show()

Let's compare it with a bias frame:

In [ ]:
bias = get_data('/Users/guido/Dropbox/courses/2019/bias/CCD Image 10.fit')
plot_medrange(bias)
plt.show()

Bias frames appear flipped along both axes (look at the dead columns), so we have to flip them again:

In [ ]:
bias = np.flip(get_data('/Users/guido/Dropbox/courses/2019/bias/CCD Image 10.fit'))
plot_medrange(bias)
plt.show()

A **master bias** is obtained by averaging several bias frames:

In [ ]:
bias_arr = np.array([np.flip(get_data('/Users/guido/Dropbox/courses/2019/bias/CCD Image %i.fit' % i)) for i in range(10,17)])
bias_med = np.median(bias_arr, axis=0)
plot_medrange(bias_med)
plt.show()

Let's remove the master bias then:

In [ ]:
sci_nobias = sci - bias_med
plot_medrange(sci_nobias[0])
plt.show()

The removal is effective in the x direction, but not in the y direction. This may happen when the bias is not stable (the CCD is not so high-quality). We don't care about this for now, as any bias residual will be removed together with the background later.

In [ ]:
plot(np.median(sci[0], axis=0))
plot(np.median(sci_nobias[0], axis=0))
plot(np.median(sci[0], axis=1))
plot(np.median(sci_nobias[0], axis=1))
plt.show()

### Make my field flat

Again, we collect and average a set of flat frames to create a **master flat**. The master bias is subtracted to the individual flat frames prior to averaging. The master flat is then normalized:

In [ ]:
flat_arr = np.array([get_data('/Users/guido/Dropbox/courses/2019/flat/CCD Image %i.fit' % i) for i in range(8,16)])
flat_med = np.median(flat_arr-bias_med, axis=0)
flat_norm = flat_med/np.mean(flat_med)
plot_medrange(flat_norm, hwid=0.05)
plt.show()

Flat-fielding is performed by dividing the debiased science frames by the master flat:

In [ ]:
sci_red = sci_nobias/flat_norm
plot_medrange(sci_red[0])
plt.show()

### Counting photons

Let's focus on the neighborhood of our target:

In [ ]:
sub_sci_red = np.array([s[450:580,580:710] for s in sci_red])
am = np.array([argmax(s) for s in sub_sci_red])
plot(sub_sci_red[0], argmax=am[0])
plot_medrange(sub_sci_red[0], argmax=am[0])
plt.show()

Let's separate target and background with a square mask. First, we use ```ogrid``` to center the indices of the image at the maximum. Then, we select the indices within a square. 

In [ ]:
def mask_sq(sci_red, am, hwidth=10):
    shape = sci_red.shape
    y, x = np.ogrid[-am[0]:shape[0]-am[0], -am[1]:shape[1]-am[1]]
    return np.logical_and(np.abs(x) < hwidth, np.abs(y) < hwidth)



inside = np.copy(sub_sci_red)  # `copy` creates new objects
outside = np.copy(sub_sci_red)
for i, o, s, a in zip(inside, outside, sub_sci_red, am):
    mask = mask_sq(s, a)
    i[~mask] = np.nan
    o[mask] = np.nan
    
plot(sub_sci_red[0], argmax=am[0])
plot(inside[0], argmax=am[0])
plot_medrange(outside[0])
plt.show()

The histograms of ```inside``` and ```outside``` are revealing:

In [ ]:
def safe(arr):
    """Reject NaNs"""
    return arr[~np.isnan(arr)]

def hist(arr, bins=None, range=None):
    """Plot histogram"""
    plt.figure()
    plt.hist(safe(arr).ravel(), bins, range, log=True) 

hist(inside[0], bins=20)
hist(outside[0], bins=20)
plt.show()

The ```outside``` image is flat enough: its median can be used as a reliable estimation of the background.

In [ ]:
med_row = np.nanmedian(outside, axis=1)
med_col = np.nanmedian(outside, axis=2)
bkg = np.array([np.nanmedian(o) for o in outside])

plot(med_row[0])
plot(med_col[0], new=False)
plt.axhline(bkg[0], c='red', linewidth=2)
plt.show()

We can finally count photons on the background-subtracted image. Note that we must convert ADUs into photons using the **gain** of the detector. The results can be plotted as a function of the Julian date.

In [ ]:
sub = np.array([i-b for i,b in zip(inside, bkg)]) 
gain = 0.6  # e-/counts
signal = np.array([np.nansum(s)*gain for s in sub])
jd = np.array([h['JD-HELIO'] for h in sci_hdr])
plot_series(jd, signal, xlabel='time (JD)')
plt.show()

Let's summarize with a new function what we've done so far. We can also roughly estimate the **signal-to-noise ratio** (assuming dark current and read-out noise are negligible): 

In [ ]:
def phot(sci_red, mask_func, gain=0.6, ron=10.6, **kwargs):
    inside = np.copy(sci_red)  
    outside = np.copy(sci_red)
    for i, o in zip(inside, outside):
        mask = mask_func(s, a, **kwargs)
        i[~mask] = np.nan
        o[mask] = np.nan
    bkg = np.array([np.nanmedian(o) for o in outside])
    sub = np.array([i-b for i,b in zip(inside, bkg)]) 
    signal = np.array([np.nansum(s)*gain for s in sub])
    snr = signal / np.sqrt((signal+2*bkg)+2*ron*2)  # Very very roughly
    return signal, snr

signal_sq, snr_sq = phot(sub_sci_red, mask_sq)
plot_series(jd, signal_sq, xlabel='time (JD)')
plot_series(jd, snr_sq, xlabel='time (JD)')
plt.show()

Remember that we used a square mask? Let's use a more suited circular mask, and shift the center a little:

In [ ]:
def mask_circ(sci_red, am, rad=11):
    shape = sci_red.shape
    y, x = np.ogrid[-am[0]:shape[0]-am[0], -am[1]:shape[1]-am[1]]
    return x**2 + y**2 < rad**2

inside = np.copy(sub_sci_red)  
outside = np.copy(sub_sci_red)
for i, o, s, a in zip(inside, outside, sub_sci_red, am):
    i[~mask] = np.nan
    o[mask] = np.nan

plot(inside[0], argmax=cen)
plot_medrange(outside[0])
plt.show()

In [ ]:
signal_circ, snr_circ = phot(sub_sci_red, mask_circ)
plot_series(jd, signal_sq, xlabel='time (JD)')
plot_series(jd, signal_circ, xlabel='time (JD)', new=False)
plot_series(jd, snr_sq, xlabel='time (JD)')
plot_series(jd, snr_circ, xlabel='time (JD)', new=False)

plt.show()